In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
!pip install datasets
!pip install evaluate
!pip install pandas

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, \
     DataCollatorWithPadding, pipeline
from datasets import Dataset
import numpy as np
import evaluate
import pandas as pd

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("/content/gdrive/My Drive/Colab Notebooks/archive/")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_file = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/archive/train_40k.csv')
unlabeled_file = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/archive/unlabeled_150k.csv')
valid_file = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/archive/val_10k.csv')

In [ ]:
dataset
sequence_labels=train_file['Cat1']
sequence_labels_test=valid_file['Cat1']
unique_sequence_labels = list(set(sequence_labels))


# add numerical label column to dataset 
id2label = {i: l for i, l in enumerate(unique_sequence_labels)}
label2id= {l: i for i, l in enumerate(unique_sequence_labels)}

sequence_labels=[label2id[x] for x in sequence_labels]
sequence_labels_test=[label2id[x] for x in sequence_labels_test]

In [ ]:
label2id
id2label

{0: 'grocery gourmet food',
 1: 'toys games',
 2: 'baby products',
 3: 'pet supplies',
 4: 'beauty',
 5: 'health personal care'}

In [ ]:
sequence_labels[0]

0

In [ ]:
dataset['train']=dataset['train'].add_column("label", sequence_labels)
dataset['validation']=dataset['validation'].add_column("label", sequence_labels_test)

In [ ]:
text = dataset.select_columns(['Text', 'label'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True)

In [ ]:
tokenized_text= text.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
tokenized_text['train'][0]


{'Text': 'The description and photo on this product needs to be changed to indicate this product is the BuffalOs version of this beef jerky.',
 'label': 0,
 'input_ids': [101,
  1109,
  6136,
  1105,
  6307,
  1113,
  1142,
  3317,
  2993,
  1106,
  1129,
  2014,
  1106,
  5057,
  1142,
  3317,
  1110,
  1103,
  139,
  9435,
  1348,
  22234,
  1683,
  1104,
  1142,
  14413,
  12935,
  1183,
  119,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer

import tensorflow as tf

batch_size = 16

num_epochs = 5

batches_per_epoch = len(tokenized_text["train"]) // batch_size

total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(

    "distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id

)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
tf_train_set = model.prepare_tf_dataset(

    tokenized_text["train"],

    shuffle=True,

    batch_size=16,

    collate_fn=data_collator,

)

tf_validation_set = model.prepare_tf_dataset(

    tokenized_text["validation"],

    shuffle=False,

    batch_size=16,

    collate_fn=data_collator,

)

In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    hub_model_id="KJangam/amznproducts",
    output_dir="amznproducts",
    tokenizer=tokenizer,

)

/content/amznproducts is already a clone of https://huggingface.co/KJangam/amznproducts. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)


In [ ]:
tf_validation_set

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>